# Benchmarking for the api - version 0

### QFT + QFT^-1

#### Probamos con AerSimulator en la interfaz de qiskit

In [24]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT

n = 10

qc = QuantumCircuit(n)

qc.x(0); qc.x(n-1); qc.x(n-2)

qc.append(QFT(n), range(n))

qc.append(QFT(n).inverse(), range(n))

qc.measure_all()

display(qc.draw())

qc = qc.decompose().decompose().decompose()

┌───┐┌──────┐┌───────┐ ░ ┌─┐                           
    q_0: ┤ X ├┤0     ├┤0      ├─░─┤M├───────────────────────────
         └───┘│      ││       │ ░ └╥┘┌─┐                        
    q_1: ─────┤1     ├┤1      ├─░──╫─┤M├────────────────────────
              │      ││       │ ░  ║ └╥┘┌─┐                     
    q_2: ─────┤2     ├┤2      ├─░──╫──╫─┤M├─────────────────────
              │      ││       │ ░  ║  ║ └╥┘┌─┐                  
    q_3: ─────┤3     ├┤3      ├─░──╫──╫──╫─┤M├──────────────────
              │      ││       │ ░  ║  ║  ║ └╥┘┌─┐               
    q_4: ─────┤4     ├┤4      ├─░──╫──╫──╫──╫─┤M├───────────────
              │  QFT ││  IQFT │ ░  ║  ║  ║  ║ └╥┘┌─┐            
    q_5: ─────┤5     ├┤5      ├─░──╫──╫──╫──╫──╫─┤M├────────────
              │      ││       │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐         
    q_6: ─────┤6     ├┤6      ├─░──╫──╫──╫──╫──╫──╫─┤M├─────────
              │      ││       │ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      
    q_7: ─────┤7     ├┤7      ├─░──╫──╫──╫──╫──╫──╫──╫─┤M├──────
         ┌───┐│      ││       │ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
    q_8: ┤ X ├┤8     ├┤8      ├─░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───
         ├───┤│      ││       │ ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
    q_9: ┤ X ├┤9     ├┤9      ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
         └───┘└──────┘└───────┘ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
meas: 10/══════════════════════════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
                                   0  1  2  3  4  5  6  7  8  9

Voy a probar con `qiskit`, usando `AerSimulator` de `qiskit_aer`.

In [12]:
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram
backend = AerSimulator()
qct = transpile(qc, backend)
result = backend.run(qct, shots = 4000).result()
counts_qiskit_aer = result.get_counts()

for b, c in counts_qiskit_aer.items():
    print(f'{b}: {c}')


1100000001: 4000


Esto es evidente: al aplicar la `QFT` y la `IQFT` deberíamos obtener el valor de entrada.

#### Probamos con la api sin ruido, con una sola qpu

En primer lugar, cargamos lo módulos y las variables de entorno que necesitamos:

In [13]:
import os
import sys
import json
import time

install_path = os.getenv("INSTALL_PATH")
print(install_path)# como no me lo pilla pues lo pongo a mano
sys.path.insert(0, "/mnt/netapp1/Store_CESGA/home/cesga/mlosada/api/api-simulator/installation")
sys.path.insert(0, "/mnt/netapp1/Store_CESGA/home/cesga/mlosada/api/api-simulator/python")

/mnt/netapp1/Store_CESGA/home/cesga/mlosada/api/api-simulator/installation


In [14]:
from qpu import QPU, getQPUs
from qjob import gather

En primer lugar usamos la función `get_QPUs()` para crear nuestra lista de máquinas.

In [15]:
qpus = getQPUs()

print("QPUs disponibles:")
for q in qpus:
    print(f"QPU {q.id_}, backend: {q.backend.name}.")

QPUs disponibles:
QPU 0, backend: BasicAer.


Vamos a obtener información sobre el backend de la primera QPU:

In [16]:
qpu0 = qpus[0]
qpu0.backend.info()

--- Backend configuration ---
basis_gates: ['u1', 'u2', 'u3', 'u', 'p', 'r', 'rx', 'ry', 'rz', 'id', 'x', 'y', 'z', 'h', 's', 'sdg', 'sx', 'sxdg', 't', 'tdg', 'swap', 'cx', 'cy', 'cz', 'csx', 'cp', 'cu', 'cu1', 'cu3', 'rxx', 'ryy', 'rzz', 'rzx', 'ccx', 'ccz', 'crx', 'cry', 'crz', 'cswap']
conditional: True
coupling_map: [[]]
custom_instructions: 
description: Usual AER simulator.
gates: []
is_simulator: True
max_shots: 10000
memory: True
n_qubits: 32
name: BasicAer
simulator: AerSimulator
url: https://github.com/Qiskit/qiskit-aer
version: 0.0.1


Probamos a mandar un `QJob` a `qpu0`, con el mismo número de shots. En primer lugar no vamos a transpilar, `transpile = False` o lo dejamos por defecto.

In [7]:
qjob0 = qpu0.run(qc, shots = 4000, transpile = True, initial_layout = [22,23,24,25,26,27,28,29,30,31])

In [8]:
result = qjob0.result()

In [9]:
from qiskit.visualization import plot_histogram
plot_histogram(result.get_counts()).savefig("histograma_fakeqmio_2.png")

In [10]:
result.get_counts()

{'0000000000': 4,
 '0000000001': 3,
 '0000010000': 4,
 '0100000000': 3,
 '0100000001': 2,
 '0100000010': 5,
 '0100000011': 3,
 '0100000100': 6,
 '0100000101': 5,
 '0100000110': 4,
 '0100000111': 4,
 '0100001000': 3,
 '0100001001': 4,
 '0100001010': 4,
 '0100001011': 2,
 '0100001100': 7,
 '0100001110': 1,
 '0100001111': 4,
 '0000010001': 8,
 '0100010000': 2,
 '0100010001': 3,
 '0100010010': 5,
 '0100010011': 5,
 '0100010100': 3,
 '0100010101': 2,
 '0100010110': 6,
 '0100010111': 4,
 '0100011000': 2,
 '0100011001': 4,
 '0100011010': 6,
 '0100011011': 5,
 '0100011100': 1,
 '0100011101': 5,
 '0100011110': 1,
 '0100011111': 2,
 '0000010010': 3,
 '0100100000': 7,
 '0100100001': 4,
 '0100100010': 3,
 '0100100011': 4,
 '0100100100': 4,
 '0100100101': 3,
 '0100100110': 3,
 '0100100111': 5,
 '0100101000': 8,
 '0100101001': 4,
 '0100101010': 3,
 '0100101011': 6,
 '0100101100': 3,
 '0100101101': 5,
 '0100101110': 3,
 '0100101111': 4,
 '0000010011': 4,
 '0100110000': 5,
 '0100110001': 5,
 '01001100

In [20]:
from qiskit import QuantumCircuit
qc = QuantumCircuit(2); qc.h(0); qc.cx(0,1); qc.measure_all()
qc.draw()

┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1

In [21]:
qjob_bell = qpus[0].run(qc, shots = 4000, transpile = True, initial_layout = [30,31])

In [22]:
result = qjob_bell.result()
from qiskit.visualization import plot_histogram
plot_histogram(result.get_counts()).savefig("histograma_fakeqmio_bell_new.png")